# Effect of Saint Sampler
(Not fully implemented yet)

This is a presentation document that tests the effects of the Saint Sampler method on the cora dataset of GRACE and GraphMAE.

First define training functions for convenience.

In [5]:
from main_GraphMAE import train_eval_whole_graph,train_eval_batch_sampler
from graphmae.utils import load_best_configs
import argparse

def build_args():
    parser = argparse.ArgumentParser(description="GAT")
    parser.add_argument("--seeds", type=int, nargs="+", default=[0])
    parser.add_argument("--dataset", type=str, default="cora")
    parser.add_argument("--device", type=int, default=-1)
    parser.add_argument("--max_epoch", type=int, default=200,
                        help="number of training epochs")
    parser.add_argument("--warmup_steps", type=int, default=-1)

    parser.add_argument("--num_heads", type=int, default=4,
                        help="number of hidden attention heads")
    parser.add_argument("--num_out_heads", type=int, default=1,
                        help="number of output attention heads")
    parser.add_argument("--num_layers", type=int, default=2,
                        help="number of hidden layers")
    parser.add_argument("--num_hidden", type=int, default=256,
                        help="number of hidden units")
    parser.add_argument("--residual", action="store_true", default=False,
                        help="use residual connection")
    parser.add_argument("--in_drop", type=float, default=.2,
                        help="input feature dropout")
    parser.add_argument("--attn_drop", type=float, default=.1,
                        help="attention dropout")
    parser.add_argument("--norm", type=str, default=None)
    parser.add_argument("--lr", type=float, default=0.005,
                        help="learning rate")
    parser.add_argument("--weight_decay", type=float, default=5e-4,
                        help="weight decay")
    parser.add_argument("--negative_slope", type=float, default=0.2,
                        help="the negative slope of leaky relu for GAT")
    parser.add_argument("--activation", type=str, default="prelu")
    parser.add_argument("--mask_rate", type=float, default=0.5)
    parser.add_argument("--drop_edge_rate", type=float, default=0.0)
    parser.add_argument("--replace_rate", type=float, default=0.0)

    parser.add_argument("--encoder", type=str, default="gat")
    parser.add_argument("--decoder", type=str, default="gat")
    parser.add_argument("--loss_fn", type=str, default="sce")
    parser.add_argument("--alpha_l", type=float, default=2, help="`pow`coefficient for `sce` loss")
    parser.add_argument("--optimizer", type=str, default="adam")

    parser.add_argument("--max_epoch_f", type=int, default=30)
    parser.add_argument("--lr_f", type=float, default=0.001, help="learning rate for evaluation")
    parser.add_argument("--weight_decay_f", type=float, default=0.0, help="weight decay for evaluation")
    parser.add_argument("--linear_prob", action="store_true", default=False)

    parser.add_argument("--load_model", action="store_true")
    parser.add_argument("--save_model", action="store_true")
    parser.add_argument("--use_cfg", action="store_true")
    parser.add_argument("--logging", action="store_true")
    parser.add_argument("--scheduler", action="store_true", default=False)
    parser.add_argument("--concat_hidden", action="store_true", default=False)

    # graph sample + mini-batch
    parser.add_argument("--use_sampler",action="store_true",default=False)
    parser.add_argument("--budget", type=int, default=500,help="number of nodes sampled per batch with SaintSampler")
    parser.add_argument("--num_iters", type=int, default=0,help="number of iters to train ( default= max_epoch*total_nodes_num/budget )")

    args = parser.parse_args(args=[])
    return args

def run_cora_test_on_graphmae(budget_list):
	args = build_args()
	args.dataset='cora'
	args = load_best_configs(args, "./configs/best_GraphMAE_configs.yml")
	args.use_sampler=True
	args.num_iters=0
	args.device=0
    
	args.encoder='gat'
	args.decoder='gat'
	acc_list=[]
	for budget in budget_list:
		args.budget=budget
		print(args)
		acc=train_eval_batch_sampler(args)
		acc_list.append(acc)
	acc_list.append(train_eval_whole_graph(args))


In [6]:
from main_Grace import train_eval_whole_graph,train_eval_batch_sampler
from grace.utils import load_best_configs
import argparse

def build_args_2():
    parser = argparse.ArgumentParser()

    parser.add_argument("--seeds", type=int, nargs="+", default=[0])
    parser.add_argument('--dataset', type=str, default='cora')
    parser.add_argument("--device", type=int, default=-1)
    parser.add_argument("--use_cfg", action="store_true")

    parser.add_argument("--encoder", type=str, default="gcn")
    parser.add_argument("--activation", type=str, default="relu")

    parser.add_argument("--drop_edge_rate_1", type=float, default=0.5)
    parser.add_argument("--drop_edge_rate_2", type=float, default=0.5)
    parser.add_argument("--drop_feature_rate_1", type=float, default=0.5)
    parser.add_argument("--drop_feature_rate_2", type=float, default=0.5)
    parser.add_argument("--tau", type=float, default=0.5)

    parser.add_argument("--num_hidden", type=int, default=256,
                        help="number of hidden units")
    parser.add_argument("--num_proj_hidden", type=int, default=256,
                        help="number of hidden units")
    parser.add_argument("--num_layers", type=int, default=2,
                        help="number of hidden layers")


    parser.add_argument("--num_heads", type=int, default=4,
                        help="number of hidden attention heads")
    parser.add_argument("--lr", type=float, default=0.005,
                        help="learning rate")
    parser.add_argument("--weight_decay", type=float, default=5e-4,
                        help="weight decay")
    parser.add_argument("--optimizer", type=str, default="adam")
    parser.add_argument("--scheduler", action="store_true", default=False)
    parser.add_argument("--max_epoch", type=int, default=200,
                        help="number of training epochs")
    parser.add_argument("--max_epoch_f", type=int, default=300)
    parser.add_argument("--lr_f", type=float, default=0.001, help="learning rate for evaluation")
    parser.add_argument("--weight_decay_f", type=float, default=0.0, help="weight decay for evaluation")
    parser.add_argument("--linear_prob", action="store_true", default=True)

    parser.add_argument("--logging", action="store_true")
    parser.add_argument("--load_model", action="store_true")
    parser.add_argument("--save_model", action="store_true")
    # graph sample + mini-batch
    parser.add_argument("--use_sampler", action="store_true", default=False)
    parser.add_argument("--budget", type=int, default=500, help="number of nodes sampled per batch with SaintSampler")
    parser.add_argument("--num_iters", type=int, default=0,
                        help="number of iters to train ( default= max_epoch*total_nodes_num/budget )")

    args = parser.parse_args(args=[])
    return args

def run_cora_test_on_grace(budget_list):
	args = build_args_2()
	args.dataset='cora'
	args = load_best_configs(args, "./configs/best_GraphMAE_configs.yml")
	args.use_sampler=True
	args.num_iters=0
	args.device=0
	args.encoder='gat'
	acc_list=[]
	for budget in budget_list:
		args.budget=budget
		print(args)
		acc=train_eval_batch_sampler(args)
		acc_list.append(acc)
	acc_list.append(train_eval_whole_graph(args))

## Then train
For fairness in comparison, the number of training sessions is set to the default value: $ N(graph_{full})\times epochs/ budget$

In [8]:
budget_list=[500,1000,1500,2000,2708]
result_graphmae=run_cora_test_on_graphmae(budget_list)
result_grace=run_cora_test_on_grace(budget_list)

2023-04-09 14:25:10,467 - INFO - Using best configs
2023-04-09 14:25:10,534 - INFO - Use schedular
2023-04-09 14:25:10,535 - INFO - start training..


------ Use best configs ------
Namespace(seeds=[0], dataset='cora', device=0, use_cfg=False, encoder='gat', activation='prelu', drop_edge_rate_1=0.5, drop_edge_rate_2=0.5, drop_feature_rate_1=0.5, drop_feature_rate_2=0.5, tau=0.5, num_hidden=512, num_proj_hidden=256, num_layers=2, num_heads=4, lr=0.001, weight_decay=0.0002, optimizer='adam', scheduler=True, max_epoch=300, max_epoch_f=300, lr_f=0.01, weight_decay_f=0.0001, linear_prob=True, logging=False, load_model=False, save_model=False, use_sampler=True, budget=500, num_iters=0, mask_rate=0.5, decoder='gat', in_drop=0.2, attn_drop=0.1, loss_fn='sce', drop_edge_rate=0.0, replace_rate=0.05, alpha_l=3)
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
####### Run 0 for seed 0


# Iters 203: train_loss: 6.4301:  12%|█▎        | 203/1624 [00:11<03:39,  6.47it/s]

# IGNORE: --- TestAcc: 0.6280, early-stopping-TestAcc: 0.6280, Best ValAcc: 0.6500 in epoch 299 --- 


# Iters 403: train_loss: 6.3536:  25%|██▍       | 403/1624 [00:23<02:41,  7.57it/s]

...

# IGNORE: --- TestAcc: 0.7550, early-stopping-TestAcc: 0.7520, Best ValAcc: 0.7440 in epoch 219 --- 


# Epoch 299: train_loss: 7.7332: 100%|██████████| 300/300 [00:26<00:00, 11.34it/s]


num parameters for finetuning: 3591


# Epoch: 299, train_loss: 0.5523, val_loss: 0.8024, val_acc:0.738, test_loss: 0.7488, test_acc: 0.7560: 100%|██████████| 300/300 [00:01<00:00, 205.00it/s]

--- TestAcc: 0.7560, early-stopping-TestAcc: 0.7560, Best ValAcc: 0.7400 in epoch 297 --- 
# final_acc: 0.7560±0.0000
# early-stopping_acc: 0.7560±0.0000


In [9]:

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

x=budget_list
y1 = result_graphmae[:-1]  # Sample data.
y2=result_grace[:-1]
plt.figure(figsize=(10, 5), layout='constrained')
plt.plot(x, y1, label='GRACE')  # Plot some data on the (implicit) axes.
plt.plot(x, y2, label='GraphMAE')  # etc.
plt.axhline(result_graphmae[-1],label='GraphMAE (full graph)')
plt.axhline(result_grace[-1],label='GRACE (full graph)')

plt.xlabel('budget')
plt.ylabel('acc')
plt.title("Effect of Saint Sampler")
plt.legend()

TypeError: 'NoneType' object is not subscriptable